# Limpieza de datos de produccion mensual agricola de la pagina del SIAP

Una vez que tengamos los datos en archivos CSV, realizaremos una pequeña exploración para llevar a cabo una limpieza sencilla. Esto implicará identificar datos faltantes, buscar instancias duplicadas y asegurarnos de que el tipo de dato de las columnas sea coherente.

### Librerías

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
def custom_to_float(value):
    '''
    Función que transformá cadenas de caracteres que son 
    números separados por comas en flotantes.
    '''
    if isinstance(value, str):
        return float(value.replace(',', ''))
    elif isinstance(value, (int, float)):
        return float(value)
    else:
        return None

In [14]:
def update_df_SIAP(file_SIAP, df_new, replace = False):
    '''
    Función para actualizar nuestra base de datos del SIAP,
    si no existe se crea con el data frame proporcionado.
    Si quieres reemplazar el archivo haz replace = True.
    '''

    if replace:
        os.remove(file_SIAP)

    if not os.path.exists(file_SIAP):
        df_new.to_csv(file_SIAP, index=False)
    else:
        df_new.to_csv(file_SIAP, mode='a', header=False, index=False)

Empezamos leyendo cada uno de los archivos y concatenándolos en un data frame

In [5]:
request_dict = {
  'anio': ['2020','2021','2022','2023'],
  'mes': ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto',
          'Septiembre', 'Octubre', 'Noviembre', 'Diciembre'],
  'ciclo': ['Ciclicos - Perennes'],
  'modalidad': ['Riego + Temporal'],
  'cultivo': ['Ajo', 'Ajonjolí', 'Alfalfa', 'Aguacate']
}

directory = '../descarga/datosSIAP'
df_temp = pd.DataFrame()

for anio in request_dict['anio']:

  for mes in request_dict['mes']:

    for ciclo in request_dict['ciclo']:

      for modalidad in request_dict['modalidad']:
        
        for cultivo in request_dict['cultivo']:    
          
          try:

            df = pd.read_csv(os.path.join(directory,(f"{anio.replace(' ', '')}_{mes.replace(' ', '')}" +
            f"_{ciclo.replace(' ', '')}_{modalidad.replace(' ', '')}" +
            f"_{cultivo.replace(' ', '')}.csv")))
          
            if df_temp.empty:
              df_temp = df
            else:
              df_temp = pd.concat([df_temp, df], axis=0)

          except Exception as e:
            print(f"Failed: {str(e)}")
            continue
            
df_temp = df_temp.reset_index(drop=True)

Failed: [Errno 2] No such file or directory: '../descarga/datosSIAP\\2023_Octubre_Ciclicos-Perennes_Riego+Temporal_Ajo.csv'
Failed: [Errno 2] No such file or directory: '../descarga/datosSIAP\\2023_Octubre_Ciclicos-Perennes_Riego+Temporal_Ajonjolí.csv'
Failed: [Errno 2] No such file or directory: '../descarga/datosSIAP\\2023_Octubre_Ciclicos-Perennes_Riego+Temporal_Alfalfa.csv'
Failed: [Errno 2] No such file or directory: '../descarga/datosSIAP\\2023_Octubre_Ciclicos-Perennes_Riego+Temporal_Aguacate.csv'
Failed: [Errno 2] No such file or directory: '../descarga/datosSIAP\\2023_Noviembre_Ciclicos-Perennes_Riego+Temporal_Ajo.csv'
Failed: [Errno 2] No such file or directory: '../descarga/datosSIAP\\2023_Noviembre_Ciclicos-Perennes_Riego+Temporal_Ajonjolí.csv'
Failed: [Errno 2] No such file or directory: '../descarga/datosSIAP\\2023_Noviembre_Ciclicos-Perennes_Riego+Temporal_Alfalfa.csv'
Failed: [Errno 2] No such file or directory: '../descarga/datosSIAP\\2023_Noviembre_Ciclicos-Perennes_R

In [6]:
df_temp

,Año,Mes,Ciclo,Modalidad,Cultivo,Entidad,Distrito,Municipio,Superficie(ha)_Sembrada,Superficie(ha)_Cosechada,Superficie(ha)_Siniestrada,Producción,Rendimiento(udm/ha)
0,2020,Enero,Ciclicos-Perennes,Riego+Temporal,Ajo,Aguascalientes,Aguascalientes,Asientos,20.0,0.0,0.0,0.0,0.0
1,2020,Enero,Ciclicos-Perennes,Riego+Temporal,Ajo,Aguascalientes,Aguascalientes,Cosío,56.0,0.0,0.0,0.0,0.0
2,2020,Enero,Ciclicos-Perennes,Riego+Temporal,Ajo,Aguascalientes,Aguascalientes,Pabellón de Arteaga,21.0,0.0,0.0,0.0,0.0
3,2020,Enero,Ciclicos-Perennes,Riego+Temporal,Ajo,Aguascalientes,Aguascalientes,Rincón de Romos,79.0,0.0,0.0,0.0,0.0
4,2020,Enero,Ciclicos-Perennes,Riego+Temporal,Ajo,Aguascalientes,Aguascalientes,Tepezalá,25.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75772,2023,Septiembre,Ciclicos-Perennes,Riego+Temporal,Aguacate,Zacatecas,Jalpa,Jalpa,2.50,0.00,0.0,0.00,0.0
75773,2023,Septiembre,Ciclicos-Perennes,Riego+Temporal,Aguacate,Zacatecas,Jalpa,Juchipila,6.50,0.00,0.0,0.00,0.0
75774,2023,Septiembre,Ciclicos-Perennes,Riego+Temporal,Aguacate,Zacatecas,Jalpa,Mezquital del Oro,6.00,0.00,0.0,0.00,0.0
75775,2023,Septiembre,Ciclicos-Perennes,Riego+Temporal,Aguacate,Zacatecas,Jalpa,Moyahua de Estrada,9.00,0.00,0.0,0.00,0.0


In [7]:
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75777 entries, 0 to 75776
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Año                         75777 non-null  int64  
 1   Mes                         75777 non-null  object 
 2   Ciclo                       75777 non-null  object 
 3   Modalidad                   75777 non-null  object 
 4   Cultivo                     75777 non-null  object 
 5   Entidad                     75777 non-null  object 
 6   Distrito                    75777 non-null  object 
 7   Municipio                   75777 non-null  object 
 8   Superficie(ha)_Sembrada     75777 non-null  object 
 9   Superficie(ha)_Cosechada    75777 non-null  object 
 10  Superficie(ha)_Siniestrada  75777 non-null  object 
 11  Producción                  75777 non-null  object 
 12  Rendimiento(udm/ha)         75777 non-null  float64
dtypes: float64(1), int64(1), object

Observamos que no tenemos ningún dato faltante. Por otro lado, la columna *Superficie(ha)_Sembrada* es de tipo 'object'; sin embargo, las columnas *Superficie(ha)_Sembrada*, *Superficie(ha)_Cosechada*, *Superficie(ha)_Siniestrada*, *Producción* y *Rendimiento(udm/ha)* deben ser de tipo numérico. La razón de esta situación es la separación por comas de los números, como se muestra en la siguiente línea:

In [8]:
df_temp[df_temp["Municipio"] == "Villa de Cos"]

,Año,Mes,Ciclo,Modalidad,Cultivo,Entidad,Distrito,Municipio,Superficie(ha)_Sembrada,Superficie(ha)_Cosechada,Superficie(ha)_Siniestrada,Producción,Rendimiento(udm/ha)
114,2020,Enero,Ciclicos-Perennes,Riego+Temporal,Ajo,Zacatecas,Zacatecas,Villa de Cos,650.0,0.0,0.0,0.0,0.00
977,2020,Enero,Ciclicos-Perennes,Riego+Temporal,Alfalfa,Zacatecas,Zacatecas,Villa de Cos,"1,250.00","1,250.00",0.0,"16,162.50",12.93
1745,2020,Febrero,Ciclicos-Perennes,Riego+Temporal,Ajo,Zacatecas,Zacatecas,Villa de Cos,650.0,0.0,0.0,0.0,0.00
2609,2020,Febrero,Ciclicos-Perennes,Riego+Temporal,Alfalfa,Zacatecas,Zacatecas,Villa de Cos,"1,250.00","1,250.00",0.0,"16,162.50",12.93
3385,2020,Marzo,Ciclicos-Perennes,Riego+Temporal,Ajo,Zacatecas,Zacatecas,Villa de Cos,810.0,0.0,0.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
71512,2023,Julio,Ciclicos-Perennes,Riego+Temporal,Alfalfa,Zacatecas,Zacatecas,Villa de Cos,"1,797.60","1,797.60",0.0,"116,772.10",64.96
72313,2023,Agosto,Ciclicos-Perennes,Riego+Temporal,Ajo,Zacatecas,Zacatecas,Villa de Cos,"1,300.00","1,300.00",0.0,"20,540.00",15.80
73313,2023,Agosto,Ciclicos-Perennes,Riego+Temporal,Alfalfa,Zacatecas,Zacatecas,Villa de Cos,"1,797.60","1,797.60",0.0,"127,719.48",71.05
74114,2023,Septiembre,Ciclicos-Perennes,Riego+Temporal,Ajo,Zacatecas,Zacatecas,Villa de Cos,"1,300.00","1,300.00",0.0,"20,540.00",15.80


Simplemente quitamos las comas de todos los números de estas columnas.

In [9]:
numeric_columns = ["Superficie(ha)_Sembrada", "Superficie(ha)_Cosechada", "Superficie(ha)_Siniestrada",
                   "Producción", "Rendimiento(udm/ha)"]
df_temp[numeric_columns] = df_temp[numeric_columns].applymap(custom_to_float)

In [10]:
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75777 entries, 0 to 75776
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Año                         75777 non-null  int64  
 1   Mes                         75777 non-null  object 
 2   Ciclo                       75777 non-null  object 
 3   Modalidad                   75777 non-null  object 
 4   Cultivo                     75777 non-null  object 
 5   Entidad                     75777 non-null  object 
 6   Distrito                    75777 non-null  object 
 7   Municipio                   75777 non-null  object 
 8   Superficie(ha)_Sembrada     75777 non-null  float64
 9   Superficie(ha)_Cosechada    75777 non-null  float64
 10  Superficie(ha)_Siniestrada  75777 non-null  float64
 11  Producción                  75777 non-null  float64
 12  Rendimiento(udm/ha)         75777 non-null  float64
dtypes: float64(5), int64(1), object

In [11]:
df_temp[df_temp["Municipio"] == "Villa de Cos"]

,Año,Mes,Ciclo,Modalidad,Cultivo,Entidad,Distrito,Municipio,Superficie(ha)_Sembrada,Superficie(ha)_Cosechada,Superficie(ha)_Siniestrada,Producción,Rendimiento(udm/ha)
114,2020,Enero,Ciclicos-Perennes,Riego+Temporal,Ajo,Zacatecas,Zacatecas,Villa de Cos,650.0,0.0,0.0,0.00,0.00
977,2020,Enero,Ciclicos-Perennes,Riego+Temporal,Alfalfa,Zacatecas,Zacatecas,Villa de Cos,1250.0,1250.0,0.0,16162.50,12.93
1745,2020,Febrero,Ciclicos-Perennes,Riego+Temporal,Ajo,Zacatecas,Zacatecas,Villa de Cos,650.0,0.0,0.0,0.00,0.00
2609,2020,Febrero,Ciclicos-Perennes,Riego+Temporal,Alfalfa,Zacatecas,Zacatecas,Villa de Cos,1250.0,1250.0,0.0,16162.50,12.93
3385,2020,Marzo,Ciclicos-Perennes,Riego+Temporal,Ajo,Zacatecas,Zacatecas,Villa de Cos,810.0,0.0,0.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
71512,2023,Julio,Ciclicos-Perennes,Riego+Temporal,Alfalfa,Zacatecas,Zacatecas,Villa de Cos,1797.6,1797.6,0.0,116772.10,64.96
72313,2023,Agosto,Ciclicos-Perennes,Riego+Temporal,Ajo,Zacatecas,Zacatecas,Villa de Cos,1300.0,1300.0,0.0,20540.00,15.80
73313,2023,Agosto,Ciclicos-Perennes,Riego+Temporal,Alfalfa,Zacatecas,Zacatecas,Villa de Cos,1797.6,1797.6,0.0,127719.48,71.05
74114,2023,Septiembre,Ciclicos-Perennes,Riego+Temporal,Ajo,Zacatecas,Zacatecas,Villa de Cos,1300.0,1300.0,0.0,20540.00,15.80


Por último, vemos que no tenemos instancias repetidas.

In [12]:
df_temp[df_temp.duplicated()]

,Año,Mes,Ciclo,Modalidad,Cultivo,Entidad,Distrito,Municipio,Superficie(ha)_Sembrada,Superficie(ha)_Cosechada,Superficie(ha)_Siniestrada,Producción,Rendimiento(udm/ha)


Ahora que estos datos están de la forma que queremos, podemos agregarlos a uno data frame general con todos los datos previamente limpiados.

In [16]:
file_SIAP = "SIAP.csv"
update_df_SIAP(file_SIAP, df_temp, replace=True)